In [20]:
import tensorflow as tf

def image_load(path):
    image = tf.io.read_file(path)
    image_tensor=tf.image.decode_jpeg(image,channels=3)
    image_tensor=tf.image.rgb_to_grayscale(image_tensor)
    image_tensor=tf.cast(image_tensor,tf.float32)
    image_tensor=tf.squeeze(image_tensor)
    image_tensor=image_tensor/255.
    return image_tensor

img = image_load('/tmp/capab/2-3g2vx6rz-0-053,jpg')
print(img.shape)
print(img)


(20, 17)
tf.Tensor(
[[0.9372549  1.         0.99607843 0.99215686 1.         0.8156863
  0.654902   0.6627451  0.83137256 0.99215686 1.         0.9882353
  0.99607843 0.9882353  0.9764706  1.         0.99215686]
 [1.         1.         0.95686275 0.9607843  1.         0.9843137
  0.8156863  0.62352943 0.49411765 0.6666667  0.83137256 0.9607843
  1.         0.99607843 0.9529412  1.         1.        ]
 [0.99607843 0.9647059  1.         1.         0.88235295 0.7529412
  0.7019608  0.654902   0.6666667  0.57254905 0.48235294 0.58431375
  0.8509804  1.         1.         0.972549   1.        ]
 [0.99607843 1.         0.9843137  0.8039216  0.6392157  0.6784314
  0.8235294  0.96862745 0.9647059  0.90588236 0.7372549  0.5372549
  0.5254902  0.7411765  0.94509804 1.         0.99215686]
 [0.9843137  1.         0.827451   0.49019608 0.654902   1.
  1.         0.9019608  0.81960785 0.9254902  1.         0.8156863
  0.47058824 0.4862745  0.70980394 0.76862746 0.93333334]
 [0.9607843  1.         0.

In [76]:
def label_to_index(label):
    alllabels = "23456789abcdefghjkmnpqrstuvwxyz"
    for i in range(len(alllabels)):
        if alllabels[i] == label:
            return float(i)

print(label_to_index('3'))
print(label_to_index('z'))

def index_to_label(idx):
    alllabels = "23456789abcdefghjkmnpqrstuvwxyz"
    return alllabels[idx]

print(index_to_label(0))
print(index_to_label(30))

1.0
30.0
2
z


In [31]:

import numpy as np

image_dataset_perelement_numpy=[]
image_dataset_labels=[]
for root, dirs, files in os.walk("/tmp/capab", topdown=False):
    image_count=len(files)
    for name in files:
        filepath = os.path.join(root, name)
        img = image_load(filepath)
        image_dataset_perelement_numpy.append(np.array(img))
        image_dataset_labels.append(label_to_index(name[0]))

image_dataset=np.array(image_dataset_perelement_numpy)
all_labels=np.array(image_dataset_labels)

print(image_count)
print(image_dataset.shape, all_labels.shape)

3284
(3284, 20, 17) (3284,)


In [32]:

test_count=int(image_count*0.2)
train_count=image_count-test_count

train_image=image_dataset[test_count:train_count]
train_label=all_labels[test_count:train_count]
test_image=image_dataset[:test_count]
test_label=all_labels[:test_count]

print(train_image.shape, train_label.shape)
print(test_image.shape, test_label.shape)

(1972, 20, 17) (1972,)
(656, 20, 17) (656,)


In [35]:
model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(20, 17)),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(31, activation='softmax')
])

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [37]:
model.fit(train_image, train_label, epochs=15)

model.evaluate(test_image,  test_label, verbose=2)


Epoch 1/15
62/62 [==============================] - 0s 1ms/step - loss: 1.1254 - accuracy: 0.8570
Epoch 2/15
62/62 [==============================] - 0s 1ms/step - loss: 0.8978 - accuracy: 0.8844
Epoch 3/15
62/62 [==============================] - 0s 1ms/step - loss: 0.7294 - accuracy: 0.9047
Epoch 4/15
62/62 [==============================] - 0s 1ms/step - loss: 0.6174 - accuracy: 0.9239
Epoch 5/15
62/62 [==============================] - 0s 1ms/step - loss: 0.5186 - accuracy: 0.9366
Epoch 6/15
62/62 [==============================] - 0s 990us/step - loss: 0.4575 - accuracy: 0.9508
Epoch 7/15
62/62 [==============================] - 0s 1ms/step - loss: 0.3936 - accuracy: 0.9594
Epoch 8/15
62/62 [==============================] - 0s 1ms/step - loss: 0.3383 - accuracy: 0.9660
Epoch 9/15
62/62 [==============================] - 0s 1ms/step - loss: 0.3157 - accuracy: 0.9660
Epoch 10/15
62/62 [==============================] - 0s 1ms/step - loss: 0.2771 - accuracy: 0.9736
Epoch 11/15
62/62

[0.10662491619586945, 0.9969512224197388]

In [39]:
model.save('capta-001.h5')


2
z


In [83]:

def topResult(result):
    topIdx = 0
    for i in range(len(result)):
        if result[i] > result[topIdx]:
            topIdx=i
    return index_to_label(topIdx)
    # print(topIdx, index_to_label(topIdx), result[topIdx])
 
procmodel = tf.keras.models.load_model("capta-001.h5")

imgdata=image_load('/tmp/capab/d-5qgxd1bh-2-680,jpg')

result=procmodel.predict(np.array([imgdata,]))

# for v in np.nditer(np.sort(result), order='C'):
#     print(v)
# print(len(np.nditer(result, order='C')))
result = result[0].tolist()
topResult(result)
# for i in range(len(result)):
#     # print(i)
#     print(i, index_to_label(i), result[i])


'd'

In [85]:
for root, dirs, files in os.walk("/tmp/capac", topdown=False):
    for name in files:
        filepath = os.path.join(root, name)
        imgdata = image_load(filepath)
        result=procmodel.predict(np.array([imgdata,]))
        result = result[0].tolist()
        newname = '%s-%s'%(topResult(result), name)
        newpath = os.path.join(root, newname)
        # print(filepath, newpath)
        os.rename(filepath, newpath)




In [87]:
def predict(imgs):
    imgdatas = []
    for img in imgs:
        imgdatas.append(image_load(img))
    result = procmodel.predict(np.array(imgdatas))
    resultStr = ''
    for i in range(len(imgs)):
        resultStr += topResult(result[i].tolist())
    return resultStr

res = predict([
    '/tmp/capac/n-sy57u_up-0-000,jpg',
    '/tmp/capac/p-sy57u_up-1-000,jpg',
    '/tmp/capac/x-sy57u_up-2-000,jpg',
    '/tmp/capac/f-sy57u_up-3-000,jpg',])
print(res)


fnpx
